## 1. Task
> **How do annual members and casual riders use Cyclistic bikes differently?**

## 2. Setup

In [ ]:
csv_files_dir <- "/kaggle/input/062022-052023-cyclistic-trip-data"
csv_files_dir

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # #
# Install required packages                 #
# tidyverse for data import and wrangling   #
# lubridate for date functions              #
# ggplot for visualization                  #
# # # # # # # # # # # # # # # # # # # # # # #
library(tidyverse)
library(lubridate)
library(ggplot2)

## 3. Load data

In [ ]:
csv_files <- dir(csv_files_dir, full.names = TRUE)

In [ ]:
csv_files

In [ ]:
# Load all data from csv files into 1 dataframe
all_trips <- do.call(rbind, lapply(csv_files, read.csv))

## 4. Clean Data

In [ ]:
# Alternative: tail(all_trips)
head(all_trips) # 6 rows

In [ ]:
colnames(all_trips)

In [ ]:
nrow(all_trips)

In [ ]:
dim(all_trips)

In [ ]:
# columns and data types (numeric, character, etc)
str(all_trips)

In [ ]:
# Ignore "start_station_name" column to "end_lng" column
all_trips <- all_trips %>%
    select(-c(start_station_name, start_station_id, end_station_name, end_station_id, start_lat, start_lng, end_lat,end_lng))

In [ ]:
# Code to convert ride_id and rideable_type to character so that they can stack correctly if program not recognize
# all_trips <- all_trips %>%
#     mutate(ride_id=as.character(ride_id)) %>%
#     mutate(rideable_type=as.character(rideable_type))

In [ ]:
# Check dataframe after transformed
# str(all_trips)

In [ ]:
head(all_trips)

In [ ]:
summary(all_trips)

In [ ]:
# How many casual and member in dataframe?
table(all_trips$member_casual)

In [ ]:
# Add columns that list the date, month, day, and year of each ride
# This will allow us to aggregate ride data for each month, day, or year ... before completing
# these operations we could only aggregate at the ride level
# https://www.statmethods.net/input/dates.html more on date formats in R found at that link
all_trips$date <- as.Date(all_trips$started_at) #The default format is yyyy-mm-dd
all_trips$month <- format(as.Date(all_trips$date), "%m")
all_trips$day <- format(as.Date(all_trips$date), "%d")
all_trips$year <- format(as.Date(all_trips$date), "%Y")
all_trips$day_of_week <- format(as.Date(all_trips$date), "%A")

In [ ]:
head(all_trips)

In [ ]:
# Calculate difference in date and time: https://stat.ethz.ch/R-manual/R-devel/library/base/html/difftime.html
all_trips$ride_length <- difftime(all_trips$ended_at, all_trips$started_at) # seconds

In [ ]:
head(all_trips)

In [ ]:
str(all_trips)

In [ ]:
# "ride_length" needs to be numeric so that can run calculations
is.numeric(all_trips$ride_length)

In [ ]:
all_trips$ride_length <- as.numeric(all_trips$ride_length)

In [ ]:
head(all_trips)

In [ ]:
str(all_trips)

In [ ]:
# The data has been processed to remove trips that are taken by staff as they service and inspect the system; 
# and any trips that were below 60 seconds in length (potentially false starts or users trying to re-dock a bike to ensure it was secure).
# Remove rows with conditions: https://www.datasciencemadesimple.com/delete-or-drop-rows-in-r-with-conditions-2/
all_trips_v2 <- all_trips[!(all_trips$ride_length <= 60),]

In [ ]:
nrow(all_trips_v2)

In [ ]:
# In case compute over and over again
# write.csv(all_trips_v2, file=file.path(getwd(), "cleaned-trip-data.csv"), row.names=FALSE)
# all_trips_v2 <- read.csv(file.path(getwd(), "cleaned-trip-data.csv"))

## 5. Descriptive Analysis

#### All figures below in seconds

In [ ]:
mean_ride_len <- mean(all_trips_v2$ride_length)
mean_ride_len

In [ ]:
# Reduce build time from kaggle, if you want to see, please uncomment or see version 4 of this release

# med_ride_len <- median(all_trips_v2$ride_length)
# med_ride_len

In [ ]:
# max_ride_len <- max(all_trips_v2$ride_length)
# max_ride_len

In [ ]:
# min_ride_len <- min(all_trips_v2$ride_length)
# min_ride_len

### All figures in format "dd hh mm ss"

In [ ]:
seconds_to_period(mean_ride_len)

In [ ]:
# Reduce build time from kaggle, if you want to see, please uncomment or see version 4 of this release

# seconds_to_period(med_ride_len)

In [ ]:
# seconds_to_period(max_ride_len)

In [ ]:
# seconds_to_period(min_ride_len)

### Aggregation Analysis

In [ ]:
# compare average riding duration (in seconds)
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN=mean) %>%
    setNames(c("User type", "Average ride time (second)"))

In [ ]:
# Reduce build time from kaggle, if you want to see, please uncomment or see version 4 of this release

# how skewness of trip duration when comparing mean
# aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN=median) %>%
#     setNames(c("User type", "Median trip duration"))

In [ ]:
# each distribution skews left or right?
# aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = max) %>%
#     setNames(c("User type", "Max trip duration"))

In [ ]:
# All the calculations (mean, median, max, min) show that trip duration 
# distribution both casual and member are skew right but casual side is more skewed.
# aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = min) %>%
#     setNames(c("User type", "Min trip duration"))

In [ ]:
# Compare average ride time by days
# aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + all_trips_v2$day_of_week, FUN=mean) %>%
#     setNames(c("User type", "Day of week", "Average trip duration"))

In [ ]:
# Order day of week for looking logically
# all_trips_v2$day_of_week <- ordered(all_trips_v2$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))

In [ ]:
# aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + all_trips_v2$day_of_week, FUN=mean) %>%
#     setNames(c("User type", "Day of week", "Average trip duration"))

In [ ]:
# Compare average ride time by bike type
# aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + all_trips_v2$rideable_type, FUN=mean) %>%
#     setNames(c("User type", "Bike type", "Average trip duration"))

In [ ]:
# Compare average trip duration by days (alternative way beside aggregate above)
# all_trips_v2 %>%
#     mutate(weekday = wday(started_at, label = TRUE)) %>% # creates weekday field using wday() - I am learning pipe and mutate :)
#     group_by(member_casual, weekday) %>% # groups by usertype and weekday
#     summarize(
#         number_of_rides=n(), # calculates the number of rides and average duration (optional)
#         average_duration = mean(ride_length) # calculates the average ride time
#     ) %>%
#     arrange(member_casual, weekday) # sorts

In [ ]:
summary(all_trips_v2$ride_length)

In [ ]:
# Make chart bigger in kaggle
options(repr.plot.width=95, repr.plot.height=45)

In [ ]:
# Average ride time between 2 groups
avg_trip_time <- all_trips_v2 %>%
    group_by(member_casual) %>%
    summarize(average_ride_time=mean(ride_length))

In [ ]:
avg_trip_time %>% 
    setNames(c("User type", "Average ride time (second)"))

In [ ]:
# https://stackoverflow.com/questions/14487188/increase-distance-between-text-and-title-on-the-y-axis
# https://stackoverflow.com/questions/12018499/how-to-put-labels-over-geom-bar-for-each-bar-in-r-with-ggplot2
# https://stackoverflow.com/questions/25061822/ggplot-geom-text-font-size-control
# https://stackoverflow.com/questions/3838774/comma-separator-for-numbers-in-r
# https://ggplot2.tidyverse.org/reference/position_dodge.html
avg_trip_time %>%
    ggplot(mapping=aes(x=member_casual, y=average_ride_time, fill=member_casual)) + 
    geom_col(position=position_dodge(0.7), width=.5) + 
    xlab("\nUser type") +
    ylab("Second\n") + 
    ggtitle("Average ride time\n") + 
    geom_text(aes(label=format(average_ride_time, big.mark=",",scientific=FALSE)), 
                position=position_dodge(width=0.9), vjust=-0.25, size=40) +
    theme(
        text= element_text(size = 80), 
        legend.key.size = unit(5, 'cm'),
        plot.title = element_text(family = 'Arial', face = 'bold', hjust = 0.5)
    )

> **Average ride time of casual riders are more than twice than member riders**

In [ ]:
# Median ride time between 2 groups
med_trip_time <- all_trips_v2 %>%
    group_by(member_casual) %>%
    summarize(med_ride_time=median(ride_length))

In [ ]:
med_trip_time %>% 
    setNames(c("User type", "Median ride time (second)"))

In [ ]:
med_trip_time %>%
    ggplot(mapping=aes(x=member_casual, y=med_ride_time, fill=member_casual)) + 
    geom_col(position=position_dodge(0.7), width=.5) + 
    xlab("\nUser type") +
    ylab("Second\n") + 
    ggtitle("Median ride time\n") + 
    geom_text(aes(label=format(med_ride_time, big.mark=",",scientific=FALSE)), 
                position=position_dodge(width=0.9), vjust=-0.25, size=40) +
    theme(
        text= element_text(size = 80), 
        legend.key.size = unit(5, 'cm'),
        plot.title = element_text(family = 'Arial', face = 'bold', hjust = 0.5)
    )

> **Median ride time of casual riders are more than nearly twice than member riders**

In [ ]:
# Avarage ride time by bike type
avg_trip_duration_by_bike_type <- all_trips_v2 %>%
    group_by(member_casual, rideable_type) %>%
    summarise(average_ride_time = mean(ride_length)) %>%
    arrange(member_casual, rideable_type)

In [ ]:
avg_trip_duration_by_bike_type %>% 
    setNames(c("User type", "Bike type","Average ride time (second)"))

In [ ]:
avg_trip_duration_by_bike_type %>%
    ggplot(aes(x = rideable_type, y = average_ride_time, fill = member_casual)) + 
    geom_col(position = position_dodge2(preserve = "single"), width=0.5) + 
    xlab("\nBike type") +
    ylab("Second\n") + 
    ggtitle("Average ride time by bike type\n") +     
    geom_text(aes(label=format(average_ride_time, big.mark=",",scientific=FALSE)), 
                position=position_dodge(width=0.9), vjust=-0.25, size=30) +
    theme(
        text= element_text(size = 80), 
        legend.key.size = unit(5, 'cm'),
        plot.title = element_text(family = 'Arial', face = 'bold', hjust = 0.5)
    )

> **Casuals have long ride time in every type of bike, longest one is docked bike and second is classic**

In [ ]:
# Total trips by day of week
no_rides_by_dow <- all_trips_v2 %>%
    mutate(weekday=wday(started_at, label = TRUE)) %>%
    group_by(member_casual, weekday) %>%
    summarize(number_of_rides=n()) %>%
    arrange(member_casual, weekday)

In [ ]:
no_rides_by_dow %>%
    setNames(c("User type", "Day", "Total trip"))

In [ ]:
# Spaces between bar see: https://www.statology.org/ggplot2-space-between-bars/
no_rides_by_dow %>%
    ggplot(mapping=aes(x=weekday, y=number_of_rides, fill=member_casual)) + 
    geom_col(position=position_dodge(0.7), width=0.65) + 
    xlab("\nDay") +
    ylab("Trip\n") + 
    ggtitle("Total trip by day\n") + 
    geom_text(aes(label=format(number_of_rides, big.mark=",",scientific=FALSE)), 
                position=position_dodge(width=0.9), vjust=-0.15, size=30) +
    theme(
        text= element_text(size = 80), 
        legend.key.size = unit(5, 'cm'),
        plot.title = element_text(family = 'Arial', face = 'bold', hjust = 0.5)
    )

> **Members take more riding trip on weekday than casuals**

In [ ]:
# Total trips by bike type
no_rides_by_bike_type <- all_trips_v2 %>%
    group_by(member_casual, rideable_type) %>%
    summarize(number_of_rides=n()) %>%
    arrange(member_casual, rideable_type)

In [ ]:
no_rides_by_bike_type %>%
    setNames(c("User type", "Bike type", "Total trip"))

In [ ]:
no_rides_by_bike_type %>%
    ggplot(aes(x = rideable_type, y = number_of_rides, fill = member_casual)) + 
    geom_col(position = position_dodge2(preserve = "single"), width=0.5) + 
    xlab("\nBike type") +
    ylab("Trip\n") + 
    ggtitle("Total trip by bike type") + 
    geom_text(aes(label=format(number_of_rides, big.mark=",",scientific=FALSE)), 
                position=position_dodge(width=0.9), vjust=-0.25, size=30) +
    theme(
        text= element_text(size = 80), 
        legend.key.size = unit(5, 'cm'),
        plot.title = element_text(family = 'Arial', face = 'bold', hjust = 0.5)
    )

> **Members take more trips by classic and eletric bike except docked bike**

In [ ]:
# Total trips by bike type and day of week
no_rides_by_bike_type_and_dow <- all_trips_v2 %>%
    mutate(weekday=wday(started_at, label = TRUE)) %>%
    group_by(member_casual, weekday, rideable_type) %>%
    summarize(number_of_rides=n()) %>%
    arrange(member_casual, weekday, rideable_type)

In [ ]:
no_rides_by_bike_type_and_dow %>%
    setNames(c("User type", "Day", "Bike type", "Total trip"))

In [ ]:
no_rides_by_bike_type_and_dow[no_rides_by_bike_type_and_dow$member_casual == 'casual',] %>%
    ggplot(aes(x = weekday, y = number_of_rides, fill = rideable_type)) + 
    geom_col(position = position_dodge2(preserve = "single"), width=0.5) + 
    xlab("\nDay") +
    ylab("Trip\n") + 
    ggtitle("Total trip of casual by bike type and day\n") + 
    geom_text(aes(label=format(number_of_rides, big.mark=",",scientific=FALSE)), 
                position=position_dodge(width=0.9), vjust=-0.25, size=30) +
    theme(
        text= element_text(size = 80), 
        legend.key.size = unit(5, 'cm'),
        plot.title = element_text(family = 'Arial', face = 'bold', hjust = 0.5)
    )

In [ ]:
no_rides_by_bike_type_and_dow[no_rides_by_bike_type_and_dow$member_casual == 'member',] %>%
    ggplot(aes(x = weekday, y = number_of_rides, fill = rideable_type)) + 
    geom_col(position = position_dodge2(preserve = "single"), width=0.5) + 
    xlab("\nDay") +
    ylab("Trip\n") + 
    ggtitle("Total trip of member by bike type and day\n") + 
    geom_text(aes(label=format(number_of_rides, big.mark=",",scientific=FALSE)), 
                position=position_dodge(width=0.9), vjust=-0.25, size=30) +
    theme(
        text= element_text(size = 80), 
        legend.key.size = unit(5, 'cm'),
        plot.title = element_text(family = 'Arial', face = 'bold', hjust = 0.5)
    )

> **Casual riders like to use electric bike most. For classic bike, casual users only ride on weekend and docked bike is the lowest compare to others**

## 6. Conclusions

By looking at average and median ride time, we can see that casual riders have greater ride time than member riders. When we investigate further, we can see that docked bike take the longest but the least total trips to be taken which makes confusion that we should focus more docked bike. The actual potention lies down electric and classic bike.

## 7. Top 3 recommendations

- More marketing campaigns focus on electric bike as a member
- Create a group or community for health or explore city rides classic bike on weekend when casual riders switch to member.
- Accumlate points when members ride on weekday for exchange Starbucks gift card, Amazon gift card,...etc
- ~~Make docked bike availble only for member.~~

## * Export Summary file for further analysis

In [ ]:
# Create a csv file that will be visualized in Excel, Tableau, or presentation software
# N.B.: This file location is for a Kaggle If you are working on a PC, change the file location
# accordingly (most likely "C:\Users\YOUR_USERNAME\Desktop\...") to export the data. For Mac user,
# location will be "~/Desktop/...."
# You can read more here: https://datatofish.com/export-dataframe-to-csv-in-r/

# Unlock this code if you want to download from Kaggle workspace or use it in local desktop
# counts = aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + all_trips_v2$day_of_week, FUN=mean)
# write_csv(counts,file="./avg_ride_length.csv")

## * Appendix

- SQL cleaning and manipulation [script](https://github.com/NguyenN95/Cyclistic-Data-Analysis/blob/main/script.sql)
- Using Google Sheet (partial data in 05/2023): https://docs.google.com/spreadsheets/d/1MVSvGrj2j5LMOvuvO2_nmV9w2frjhnV27fHL112lSzA/
- Dataset (12 months): https://www.kaggle.com/datasets/nguyenn95/062022-052023-cyclistic-trip-data
- Tableau: [story](https://public.tableau.com/app/profile/nguyen.nguyen6937/viz/Cyclisticbikeusageanalysis/HowdoannualmembersandcasualridersuseCyclisticbikesdifferently)